In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import smooth from O2_FEV1_analysis
import sys
sys.path.append("../O2_FEV1_analysis")
import smooth

# Import biology module
import sys

sys.path.append("../data")
import biology as bio
import breathe_data


plotsdir = "../../../../PlotsSmartcare/O2_FEV1/"


In [2]:
df = breathe_data.build_O2_FEV1_df()


*** Building O2 Saturation and FEV1 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
Number of IDs:  233
Number of rows:  26812
Number of FEV1 recordings: 23778
Number of O2 Saturation recordings: 23431
Dropped 0 entries with NaN O2 Saturation and NaN FEV1
Dropped 3381 entries with NaN O2 Saturation
Dropped 3034 entries with NaN FEV1
20397 entries remain
Warning for ID 113: FEV1 % Predicted should be in 0-120%, got 125.39378062710472
Warning for ID 113: FEV1 % Predicted should be in 0-120%, got 132.27514663712876
Warning for ID 113: FEV1 % Predicted should be in 0-120%, got 131.2556850060141
Warning for ID 113: FEV1 % Predicted should be in 0-120%, got 122.33539573

In [3]:
# Remove entries where less than 10 datapoints for ID
df = df.groupby("ID").filter(lambda x: len(x) > 10)
df = smooth.smooth(df, ["FEV1"])
df["FEV1 % Predicted Smoothed"] = df["FEV1 smoothed"] / df["Predicted FEV1"] * 100
df.head()

/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2_modelling/../O2_FEV1_analysis/smooth.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  O2_FEV1["{} smoothed".format(var)][mask] = smooth_vector(
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2_modelling/../O2_FEV1_analysis/smooth.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  O2_FEV1["{} smoothed".format(var)][mask] = smooth_vector(
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2_modelling/../O2_FEV1_analysis/smooth.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

,ID,Date Recorded,FEV1,O2 Saturation,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,FEV1 % Predicted,O2 Saturation % Healthy,FEV1 smoothed,FEV1 % Predicted Smoothed
0,101,2019-02-20,1.31,97.0,53,Male,173.0,3.610061,97.397202,36.287474,99.592183,1.68,46.536607
1,101,2019-02-21,1.29,96.0,53,Male,173.0,3.610061,97.397202,35.733466,98.565459,1.32,36.564477
2,101,2019-02-22,1.32,96.0,53,Male,173.0,3.610061,97.397202,36.564477,98.565459,1.32,36.564477
3,101,2019-02-23,1.28,97.0,53,Male,173.0,3.610061,97.397202,35.456463,99.592183,1.33,36.841481
4,101,2019-02-24,1.33,98.0,53,Male,173.0,3.610061,97.397202,36.841481,100.618906,1.36,37.672492


In [14]:
# Plot FEV1 % Predicted with time for individual 122
def plot_fev1_o2(df, ids):
    for id in ids:
        df_for_ID = df[df.ID == id]
        # Create subplot with 2 rows
        fig = make_subplots(rows=2, cols=1)
        # Add trace for FEV1 % Predicted on one subplot
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID["FEV1 % Predicted Smoothed"],
                mode="lines+markers",
                name="FEV1 % Predicted Smoothed",
            ),
            row=1,
            col=1,
        )
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID["FEV1 % Predicted"],
                mode="markers",
                name="FEV1 % Predicted",
            ),
            row=1,
            col=1,
        )
        # Add trace for O2 Saturation on another subplot
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID["O2 Saturation"],
                mode="markers",
                name="O2 Saturation",
            ),
            row=2,
            col=1,
        )
        fig.update_traces(marker=dict(size=3), line=dict(width=0.5))
        title = f"FEV1 % Predicted for individual {id} ({len(df_for_ID)} datapoints)"
        fig.update_layout(title=title)
        # Add trace for O2 Saturation on another subplot

        fig.show()


In [5]:
df.describe()

,FEV1,O2 Saturation,Age,Height,Predicted FEV1,Healthy O2 Saturation,FEV1 % Predicted,O2 Saturation % Healthy,FEV1 smoothed,FEV1 % Predicted Smoothed
count,20209.000000,20209.000000,20209.000000,20209.000000,20209.000000,20209.000000,20209.000000,20209.000000,20209.000000,20209.000000
mean,2.194594,96.965620,34.849720,166.222574,3.505039,97.776606,62.748645,99.171016,2.261515,64.672600
std,0.812933,1.648975,10.155274,9.148371,0.648687,0.349272,20.264082,1.674614,0.819533,20.303227
min,0.490000,76.000000,18.000000,143.000000,2.213968,97.229862,15.013975,77.518494,0.500000,15.320382
25%,1.550000,96.000000,27.000000,160.000000,2.979444,97.397202,46.531510,98.556993,1.610000,48.198629
50%,2.030000,97.000000,34.000000,166.000000,3.386997,98.011836,61.235539,99.609297,2.090000,62.880920
75%,2.760000,98.000000,41.000000,173.000000,3.987357,98.099689,75.593196,100.009268,2.820000,77.606116
max,5.260000,100.000000,64.000000,193.000000,5.322753,98.241928,149.505350,102.787144,5.260000,149.505350


In [6]:
# 0% airway resistance: FEV1 = Predicted FEV1
# 25% airway resistance: FEV1 = 0.75 * Predicted FEV1
# Negative airway resistance: FEV1 > Predicted FEV1
# Airway resistance = 1 - FEV1/Predicted FEV1
df["Airway Resistance (%)"] = (1 - df.FEV1 / df["Predicted FEV1"]) * 100
df["Airway Resistance Smoothed (%)"] = 100 - df["FEV1 % Predicted Smoothed"]

df["O2 Sat / Healthy O2 Sat (%)"] = (df["O2 Saturation"] / df["Healthy O2 Saturation"]) * 100

df.head()

,ID,Date Recorded,FEV1,O2 Saturation,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,FEV1 % Predicted,O2 Saturation % Healthy,FEV1 smoothed,FEV1 % Predicted Smoothed,Airway Resistance (%),Airway Resistance Smoothed (%),O2 Sat / Healthy O2 Sat (%)
0,101,2019-02-20,1.31,97.0,53,Male,173.0,3.610061,97.397202,36.287474,99.592183,1.68,46.536607,63.712526,53.463393,99.592183
1,101,2019-02-21,1.29,96.0,53,Male,173.0,3.610061,97.397202,35.733466,98.565459,1.32,36.564477,64.266534,63.435523,98.565459
2,101,2019-02-22,1.32,96.0,53,Male,173.0,3.610061,97.397202,36.564477,98.565459,1.32,36.564477,63.435523,63.435523,98.565459
3,101,2019-02-23,1.28,97.0,53,Male,173.0,3.610061,97.397202,35.456463,99.592183,1.33,36.841481,64.543537,63.158519,99.592183
4,101,2019-02-24,1.33,98.0,53,Male,173.0,3.610061,97.397202,36.841481,100.618906,1.36,37.672492,63.158519,62.327508,100.618906


In [7]:
# Plot Airway resistance vs O2 drop
title = f"Airway Resistance vs O2 drop ({df.ID.nunique()} IDs, {len(df)} datapoints)"
fig = px.scatter(df, x="Airway Resistance (%)", y="O2 Sat / Healthy O2 Sat (%)", title=title)
# Reduce marker size
fig.update_traces(marker=dict(size=2))
fig.show()
# 100% is about 98 for females and 97.4 for males
# Hence, threshold at 95% => 3-3.5% drop in O2 Saturation => 96.5-97% O2 Saturation

In [8]:
# Plot Airway resistance vs O2 drop
title = f"Airway Resistance vs O2 drop ({df.ID.nunique()} IDs, {len(df)} datapoints)"
fig = px.scatter(df, x="Airway Resistance Smoothed (%)", y="O2 Sat / Healthy O2 Sat (%)", title=title)
# Reduce marker size
fig.update_traces(marker=dict(size=2))
fig.show()
# 100% is about 98 for females and 97.4 for males
# Hence, threshold at 95% => 3-3.5% drop in O2 Saturation => 96.5-97% O2 Saturation

## Low airway resistance

In [9]:
# Filter airway resistance below 40%
df[df["Airway Resistance (%)"] < -20]
# '113', '126', '202', '331'

,ID,Date Recorded,FEV1,O2 Saturation,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,FEV1 % Predicted,O2 Saturation % Healthy,FEV1 smoothed,FEV1 % Predicted Smoothed,Airway Resistance (%),Airway Resistance Smoothed (%),O2 Sat / Healthy O2 Sat (%)
2783,113,2020-11-24,4.92,98.0,26,Male,165.5,3.923640,97.459955,125.393781,100.554120,5.19,132.275147,-25.393781,-32.275147,100.554120
2784,113,2020-12-07,5.19,99.0,26,Male,165.5,3.923640,97.459955,132.275147,101.580182,5.19,132.275147,-32.275147,-32.275147,101.580182
2785,113,2020-12-08,5.15,98.0,26,Male,165.5,3.923640,97.459955,131.255685,100.554120,5.19,132.275147,-31.255685,-32.275147,100.554120
2789,113,2020-12-14,4.80,99.0,26,Male,165.5,3.923640,97.459955,122.335396,101.580182,4.82,122.845127,-22.335396,-22.845127,101.580182
2790,113,2020-12-15,4.82,98.0,26,Male,165.5,3.923640,97.459955,122.845127,100.554120,4.86,123.864588,-22.845127,-23.864588,100.554120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13689,202,2020-09-11,3.00,99.0,64,Female,157.0,2.213968,98.124790,135.503338,100.891935,3.00,135.503338,-35.503338,-35.503338,100.891935
13691,202,2020-09-25,3.00,99.0,64,Female,157.0,2.213968,98.124790,135.503338,100.891935,3.31,149.505350,-35.503338,-49.505350,100.891935
13692,202,2020-10-02,3.31,98.0,64,Female,157.0,2.213968,98.124790,149.505350,99.872825,3.31,149.505350,-49.505350,-49.505350,99.872825
13720,202,2021-04-23,2.80,98.0,64,Female,157.0,2.213968,98.124790,126.469782,99.872825,2.80,126.469782,-26.469782,-26.469782,99.872825


In [20]:
plot_fev1_o2(df, ['113', '126', '202', '331'])

## High airway resistance

In [15]:
# Filter airway resistance below 40%
df[df["Airway Resistance (%)"] > 80]
# 3 individuals '122', '198', '286' have airway resistance > 80%

,ID,Date Recorded,FEV1,O2 Saturation,Age,Sex,Height,Predicted FEV1,FEV1 % Predicted,Healthy O2 Saturation,Airway resistance,Relative O2 drop (%)
4227,122,2019-08-18,0.79,96.0,37,Male,171.0,3.950904,19.995423,97.413936,80.004577,98.548528
4247,122,2020-01-21,0.79,97.0,37,Male,171.0,3.950904,19.995423,97.413936,80.004577,99.575075
4258,122,2020-04-20,0.74,98.0,37,Male,171.0,3.950904,18.729890,97.413936,81.270110,100.601622
13118,198,2020-09-07,0.53,95.0,36,Female,151.0,2.657664,19.942328,98.174992,80.057672,96.765987
13150,198,2020-10-24,0.51,91.0,36,Female,151.0,2.657664,19.189787,98.174992,80.810213,92.691629
...,...,...,...,...,...,...,...,...,...,...,...,...
18495,286,2021-06-04,0.50,90.0,42,Female,170.0,3.263626,15.320382,98.016019,84.679618,91.821725
18496,286,2021-06-12,0.51,90.0,42,Female,170.0,3.263626,15.626790,98.016019,84.373210,91.821725
18497,286,2021-06-20,0.50,90.0,42,Female,170.0,3.263626,15.320382,98.016019,84.679618,91.821725
18498,286,2021-06-24,0.50,90.0,42,Female,170.0,3.263626,15.320382,98.016019,84.679618,91.821725


In [16]:
# Plot FEV1 % Predicted with time for individual 122
plot_fev1_o2(df, ["122", "198", "286"])

## High O2 Drop

In [19]:
df[df["O2 Sat / Healthy O2 Sat (%)"] < 90].ID.unique()

array(['111', '180', '352'], dtype=object)

In [18]:
plot_fev1_o2(df, ['111', '180', '352'])